<img src="images/OFE-color-horizontal.png" width="150"> &emsp;&emsp; 
<img src="images/OMSF-logo-horizontal-color.png" width="200"> 
# Open Free Energy: An Open Source Ecosystem for Calculating Free Energies

# 0. Overview

## The ``Hello, World!`` of Free Energy Calculations

The OpenFE ecosystem is an open-source framework for calculating alchemical free energies.

In this demo, we will demonstrate how you can use either the CLI to execute physics-based simulations and predict the relative binding free energy (RBFE) of a ligand binding to a protein.

Specifically, we will be predicting which ligand from a set of candidates binds best to a target protein. We will be using TYK2 (Tyrosine kinase 2) as our target protein, which can be thought of as the "Hello, World" of free energy calculations.

For convenience, a prepared PDB structure of the
TYK2 protein is provided in `inputs/tyk2_protein.pdb`:

<img src="images/tyk2.png" width="250" alt="TYK2 protein"> &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 
<img src="images/tmp_network.png" width="250"> 

OpenFE RBFE calculations can be thought of as 3 distinct steps:

 1. **Setup**: construct a graph relating the ligands
 2. **Run**: execute a physics-based simulation
 3. **Gather**: compute meaningful metrics from the simulation data

Each stage is supported by the OpenFE software packages, but everything can be accessed from the base **openfe** package.

<img src="images/ecosystem.png" width="450">

**NOTE**: OpenFE's RBFE protocol uses [OpenMM](https://openmm.org/) as the physics based simulation engine.
<!-- **NOTE**: OpenFE also provides protocols for running [Absolute Hydration Free Energy Calculations](https://docs.openfree.energy/en/latest/tutorials/ahfe_tutorial.html) and [Molecular Dynamics (MD) simulations](https://docs.openfree.energy/en/latest/tutorials/md_tutorial.html). 
Also, see our [tutorials](https://docs.openfree.energy/en/stable/tutorials/index.html) for step-by-step guides. -->

# 0. Setup for Google Colab

If you are running this example in Google Colab, run the following cells to setup the environment.\
If you are running this notebook locally, skip down to `1. Relative Binding Free Energies with the **openfe** CLI`

In [1]:
# NBVAL_SKIP
# Only run this cell if on google colab
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"  # hack for google colab, not needed for local execution

if "COLAB_RELEASE_TAG" in os.environ:
    !pip install -q condacolab
    import condacolab
    condacolab.install_from_url("https://github.com/OpenFreeEnergy/openfe/releases/download/v1.6.0/OpenFEforge-1.6.0-Linux-x86_64.sh")

In [2]:
# NBVAL_SKIP
# Only run this cell if on google colab
import os
if "COLAB_RELEASE_TAG" in os.environ:
    import condacolab
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
    !mkdir inputs && cd inputs && openfe fetch rbfe-tutorial
    for _ in range(3):
      # Sometimes we have to re-run the check
      try:
        condacolab.check()
      except:
        pass
      else:
        break

# 1. Relative Binding Free Energies with the **openfe** CLI

First, we'll show how you can run a basic RBFE campaign with no Python at all!

The 3 stages described above each correspond to a CLI command:

**1. setup**:``openfe plan-rbfe-network``

**2. run** ``openfe quickrun``

**3. gather** ``openfe gather``

All you need is your target protein in `.pdb` format, and your candidate ligands in `.sdf` format.

## 1.1 Setup

The `openfe plan-rbfe-network` command plans a relative binding free energy network and saves it as a series of JSON files for the `openfe quickrun` command to process.

Since the CLI tool is a wrapper intended for the simplest cases, it chooses pratical defaults so that you don't have to define them. 
<!-- # TODO: add link to defaults -->

These choices can be customized by creating a settings yaml file, which is
passed in via the ``-s settings.yaml`` option. For more details, please visit our user guide section about [Customising CLI planning with YAML settings](https://docs.openfree.energy/en/latest/guide/cli/cli_yaml.html)

In [1]:
# note, we use ligands with charges already assigned to speed up this step, but this is optional
! openfe plan-rbfe-network -M inputs/tyk2_ligands_charged.sdf -p inputs/tyk2_protein.pdb -o tyk2_campaign

RBFE-NETWORK PLANNER
______________________

/Users/atravitz/micromamba/envs/openfe/lib/python3.12/site-packages/openff/utilities/provenance.py:25: CondaExecutableNotFoundWarning: No conda/mamba/micromamba executable found. Unable to determine package versions.
  warnings.warn(
Parsing in Files: 
	Got input: 
		Small Molecules: SmallMoleculeComponent(name=lig_ejm_31) SmallMoleculeComponent(name=lig_ejm_42) SmallMoleculeComponent(name=lig_ejm_46) SmallMoleculeComponent(name=lig_ejm_43) SmallMoleculeComponent(name=lig_ejm_47) SmallMoleculeComponent(name=lig_ejm_50) SmallMoleculeComponent(name=lig_jmc_23) SmallMoleculeComponent(name=lig_ejm_48) SmallMoleculeComponent(name=lig_jmc_27) SmallMoleculeComponent(name=lig_jmc_28)
		Protein: ProteinComponent(name=)
		Cofactors: []
		Solvent: SolventComponent(name=O, Na+, Cl-)

Using Options:
	Mapper: <LomapAtomMapper (time=20, threed=True, max3d=1.0, element_change=True, seed='', shift=False)>
	Mapping Scorer: <function default_lomap_score at 0x1

### visualizing the network

We can visualize the network stored in `ligand_network.graphml` using the `openfe view-ligand-network` command:

In [2]:
! ls tyk2_campaign/

ligand_network.graphml transformations        tyk2_campaign.json


In [3]:
# !openfe view-ligand-network tyk2_campaign/ligand_network.graphml

## 1.2 Run

You can run each leg individually by using the `openfe quickrun` command. It takes a transformation JSON as input, and the flags `-o` to give the final output JSON file and `-d` for the directory where simulation results should be stored. For example,

```bash
openfe quickrun tyk2_json/lig_ejm_31_lig_ejm_47_complex.json -o results_complex.json -d working-directory
openfe quickrun tyk2_json/lig_ejm_31_lig_ejm_47_solvent.json -o results_solvent.json -d working-directory
```

Molecular simulation is time-intensive, so instead of running on high-performance compute, you can copy down example results to explore the next step:

In [4]:
# Results from our cli tutorial
!openfe fetch rbfe-tutorial-results
# Extract results
!tar -xf rbfe_results.tar.gz

Fetching /Users/atravitz/micromamba/envs/openfe/lib/python3.12/site-packages/openfecli/tests/data/rbfe_results.tar.gz


## 1.3 Gather

Once the simulations are complete, you can use `openfe gather` to calculate the $\Delta G$ values for each ligand:

In [5]:
!openfe gather results/ --report raw

/Users/atravitz/micromamba/envs/openfe/lib/python3.12/site-packages/openff/utilities/provenance.py:25: CondaExecutableNotFoundWarning: No conda/mamba/micromamba executable found. Unable to determine package versions.
  warnings.warn(
┌─────────┬────────────┬────────────┬─────────────────────┬────────────────────┐
│         │            │            │                     │ MBAR uncertainty   │
│ leg     │ ligand_i   │ ligand_j   │ DG(i->j) (kcal/mol) │ (kcal/mol)         │
├─────────┼────────────┼────────────┼─────────────────────┼────────────────────┤
│ complex │ lig_ejm_31 │ lig_ejm_42 │ -14.8               │ 0.8                │
│ complex │ lig_ejm_31 │ lig_ejm_42 │ -14.9               │ 0.8                │
│ complex │ lig_ejm_31 │ lig_ejm_42 │ -15.1               │ 0.8                │
│ solvent │ lig_ejm_31 │ lig_ejm_42 │ -15.7               │ 0.8                │
│ solvent │ lig_ejm_31 │ lig_ejm_42 │ -15.7               │ 0.8                │
│ solvent │ lig_ejm_31 │ lig_ejm_42 │

In [6]:
!openfe gather results/ 

/Users/atravitz/micromamba/envs/openfe/lib/python3.12/site-packages/openff/utilities/provenance.py:25: CondaExecutableNotFoundWarning: No conda/mamba/micromamba executable found. Unable to determine package versions.
  warnings.warn(
┌────────────┬────────────────────┬────────────────────────┐
│ ligand     │ DG(MLE) (kcal/mol) │ uncertainty (kcal/mol) │
├────────────┼────────────────────┼────────────────────────┤
│ lig_ejm_31 │ -0.09              │ 0.05                   │
│ lig_ejm_42 │ 0.7                │ 0.1                    │
│ lig_ejm_46 │ -0.98              │ 0.05                   │
│ lig_ejm_47 │ -0.1               │ 0.1                    │
│ lig_ejm_48 │ 0.53               │ 0.09                   │
│ lig_ejm_50 │ 0.91               │ 0.06                   │
│ lig_ejm_43 │ 2.0                │ 0.2                    │
│ lig_jmc_23 │ -0.68              │ 0.09                   │
│ lig_jmc_27 │ -1.1               │ 0.1                    │
│ lig_jmc_28 │ -1.25              

# 2. Customization with `settings.yaml`

## 2.1 Exploring atom mappings

## x.x: Setup


### 2.1.2: Ligand Atom Mapping

In the [RBFE protocol](https://docs.openfree.energy/en/latest/guide/protocols/relativehybridtopology.html) , an atom mapping defines which atoms are mutated during the alchemical transformation.
The user can choose between two different atoms:
1. `LomapAtomMapper`: based on the maximum common substructure (MCS)
2. `KartografAtomMapper`: based on the 3D geometries of the ligands

While we use the defaults here, please note that any supported arguments of Lomap and Kartograf can be passed to the atom mapper.

**1. `LomapAtomMapper`** 

In [17]:
from openfe.setup import LomapAtomMapper
mapper = LomapAtomMapper()
lomap_mapping = next(mapper.suggest_mappings(ligand_mols[0], ligand_mols[4]))

NameError: name 'ligand_mols' is not defined

We can also visualize the atom mappings by invoking the individual OpenFE `AtomMapping` objects directly.

Unique atoms between each mapping are shown in red, and atoms which are mapped but undergo element changes are shown in blue. Bonds which either involve atoms that are unique or undergo element changes are highlighted in red.

In [ ]:
# We can display the atom mapping in 2D by calling it
lomap_mapping

It is also possible to visualize the mapping in 3D using py3dmol:

Here, the ``visualization_3D`` method displays the two end state molecules (left and right), in addition to the hybrid molecule (middle).

Atoms that have the same sphere color in both end states are mapped (i.e. will be interpolated between each other), whilst those that do not have a coloured sphere are unmapped (i.e. will be transformed into dummy atoms in the opposite end state).

In [ ]:
# Visualize the mapping in 3D
from openfe.utils import visualization_3D

visualization_3D.view_mapping_3d(lomap_mapping, show_atomIDs=True)

**2. `KartografAtomMapper`**

We can also use the `KartografAtomMapper` which is based on the 3D geometries of the ligands.

In [ ]:
from kartograf import KartografAtomMapper
# Build Kartograf Atom Mapper
mapper = KartografAtomMapper(atom_map_hydrogens=True)

# Get Mapping
kartograf_mapping = next(mapper.suggest_mappings(ligand_mols[0], ligand_mols[4]));

In [ ]:
# We can display the atom mapping in 2D by calling it
kartograf_mapping

In [ ]:
# Visualize the mapping in 3D
from openfe.utils import visualization_3D

visualization_3D.view_mapping_3d(kartograf_mapping, show_atomIDs=True)

### 2.1.3: Creating a ligand network


A `LigandNetwork` is a set of `SmallMoleculeComponent`s that are connected by `AtomMapping`s of two small molecules. 

The user can choose between multiple different network topologies:
* Minimial spanning tree (MST)
* LOMAP network
* Radial (star) network
* Loading in networks from external software (FEP+ or Orion)
* Loading in a user-defined network

In this section, we will create and visualize the MST, LOMAP, and radial networks for the TYK2 dataset.

Here, we will be using the `LomapAtomMapper` as the atom mapper for all networks.

In [ ]:
# Create network from the two molecules
import openfe
from openfe.setup.ligand_network_planning import generate_radial_network
from openfe.setup.ligand_network_planning import generate_minimal_spanning_network
from openfe.setup.ligand_network_planning import generate_lomap_network
from openfe.setup import LomapAtomMapper

# Create an MST network
mst_network = generate_minimal_spanning_network(
    ligands=ligand_mols,
    scorer=openfe.lomap_scorers.default_lomap_score,
    mappers=[LomapAtomMapper(),]);

# Create a LOMAP network
lomap_network = generate_lomap_network(
    ligands=ligand_mols,
    scorer=openfe.lomap_scorers.default_lomap_score,
    mappers=[LomapAtomMapper(),]);

We can plot out the different networks to visualize their structure and to see how ligands are being transformed.

In [ ]:
from konnektor.visualization import draw_ligand_network
import matplotlib.pyplot as plt
fig, ax = plt.subplots(ncols=2, figsize=(20,10), dpi=120, squeeze=True)
draw_ligand_network(mst_network,node_size=4400, fontsize=26, title="Minimal Spanning Tree",ax=ax[0]);
draw_ligand_network(lomap_network,node_size=4400, fontsize=26, title="LOMAP network", ax=ax[1]);
fig.tight_layout()

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

# Extract the contents of the sdf file and visualise it
ligands_rdmol = [mol for mol in
                 Chem.SDMolSupplier('inputs/tyk2_ligands.sdf', removeHs=False)]

for ligand in ligands_rdmol:
    AllChem.Compute2DCoords(ligand)

Chem.Draw.MolsToGridImage(ligands_rdmol, molsPerRow=5)

Edges along the network can be accessed to recover the individual molecules involved in a given alchemical tranformation and the atom mapping between the two ligands. 

**Note: as can be seen in the example below, transformations are defined within OpenFE as going from componentA to componentB**

In [ ]:
mst_edges = [edge for edge in mst_network.edges]

# Pick an edge
edge = mst_edges[1]

# Print the smiles of the molecules and the mapping
print("molecule A smiles: ", edge.componentA.smiles)
print("molecule B smiles: ", edge.componentB.smiles)
print("map between molecule A and B: ", edge.componentA_to_componentB)

In [ ]:
# We can display the atom mapping of an edge by calling it
edge

In [ ]:
from IPython.display import Image

# mappings can also be saved to file if required
edge.draw_to_file('tyk2_edge.png')

# load it back for visualisation
Image("tyk2_edge.png")

#### Storing the ligand network

Created networks can easily be converted to (and also loaded from) a GraphML representation.

This can allow users of OpenFE to store the network to disk for later use.

In [ ]:
# Convert to graphml
with open("network_store.graphml", "w") as writer:
    writer.write(mst_network.to_graphml())

In [ ]:
# First let's define the Protein and Solvent Components which we will be using
from openfe import SolventComponent, ProteinComponent
from openff.units import unit

protein = ProteinComponent.from_pdb_file('inputs/tyk2_protein.pdb')

# Note: the distance from the solute to add water is not defined here but in the
# the relevant RBFE solver method
solvent = SolventComponent(positive_ion='Na', negative_ion='Cl',
                           neutralize=True, ion_concentration=0.15*unit.molar)

In [ ]:
# Extract the relevant edge for the lig_ejm_31 -> lig_ejm_47 transform in the radial graph
ejm_31_to_ejm_47 = [edge for edge in mst_network.edges if edge.componentB.name == "lig_ejm_47"][0]

ejm_31_to_ejm_47

We’ll write out the transformation to disk, so that it can be run using the `openfe quickrun` command:

In [ ]:
import pathlib
# first we create the directory
transformation_dir = pathlib.Path("tyk2_json")
transformation_dir.mkdir(exist_ok=True)

# then we write out the transformations
transformation_complex.dump(transformation_dir / f"{transformation_complex.name}.json")
transformation_solvent.dump(transformation_dir / f"{transformation_solvent.name}.json")

You can run the RBFE simulations from the CLI by using the `openfe quickrun` command, as described in Section 5. below.

# 3. Useful References for Exploring OpenFE Further

In our [documentation](https://docs.openfree.energy/en/latest/index.html), 
we provide tutorials for ever protocol to walk you through setup, execution and analysis step by step.

* [RBFE CLI tutorial](https://docs.openfree.energy/en/latest/tutorials/rbfe_cli_tutorial.html)
* [RBFE Python tutorial](https://docs.openfree.energy/en/latest/tutorials/rbfe_python_tutorial.html)
* [AHFE tutorial](https://docs.openfree.energy/en/latest/tutorials/ahfe_tutorial.html)
* [MD tutorial](https://docs.openfree.energy/en/latest/tutorials/md_tutorial.html)

In addition to the tutorials, you can find [cookbooks](https://docs.openfree.energy/en/latest/cookbook/index.html), written as How-to guides on how to utilize different components of the toolkit, as well as a [User Guide](https://docs.openfree.energy/en/latest/guide/index.html) that goes into the underlying concepts of the OpenFE toolkit.

For details about the toolkit's core methods and classes, please visit our [API Reference](https://docs.openfree.energy/en/latest/reference/index.html) or our [Github page](https://github.com/OpenFreeEnergy/openfe).

To learn more about the project, our team and how you can get involved, please visit our [Homepage](https://openfree.energy/) or get in touch at OpenFreeEnergy@omsf.io.